In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)
# cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
cwd_parent = os.path.abspath(os.path.join(cwd, '../../'))
print(cwd_parent)

sys.path.append(cwd_parent)

/opt/data/chemprop_run/git/notebooks/AltModels
/opt/data/chemprop_run/git


In [5]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelper import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
from ogb.graphproppred import Evaluator

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 1
cuda:0, name:NVIDIA GeForce GTX 1080 Ti
total memory available: 10.91656494140625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB



In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

1

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


### Preparing dataset 

In [28]:
score = 'total_thresh'
score_val = 4

In [30]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

# v_1: GNN
# v_2: Alt Models (Baseline)
data_fname = 'data_v2'

In [31]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# print(targetdata_dir)

path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
path_current_dir /opt/data/chemprop_run/git/deepadr
/opt/data/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v2


In [32]:
xFlat = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'X_flat.pkl'))
y = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'y.pkl'))
expression = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'expression.pkl'))

In [33]:
xFlatMat = torch.stack([torch.cat(i) for i in list(xFlat.values())])
xFlatMat.shape

torch.Size([25757, 18])

In [34]:
torch.tensor(expression)

tensor([[ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        [ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        [ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        ...,
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537],
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537],
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537]],
       dtype=torch.float64)

In [35]:
x = torch.cat([xFlatMat, torch.tensor(expression)], dim=1)
x.shape

torch.Size([25757, 926])

In [45]:
y

array([0, 0, 0, ..., 0, 1, 1])

In [47]:
fold_partitions = get_stratified_partitions(y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [51]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


### Alt Models

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
# from sklearn.inspection import permutation_importance

In [53]:
x_np = x.numpy()
y_np = y

In [54]:
scaler = StandardScaler()
x_np_scale = scaler.fit_transform(x_np)

In [58]:
targetdata_dir_exp

'/opt/data/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v2/experiments'

In [59]:
model = LogisticRegression(max_iter=10000,
                                      random_state=42,
                                      solver="lbfgs",
                                      penalty='l2',
                                      l1_ratio=None,
                                      verbose=1)

In [60]:
# model = DecisionTreeClassifier()

In [61]:
# model = KNeighborsClassifier()

In [62]:
# model = DecisionTreeClassifier()

In [63]:
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("Start: " + time_stamp)

for q_i in range(len(fold_partitions)):
    partition = fold_partitions[q_i]
    
    ids_train = partition['train']
    ids_test = partition['test']
    
    part_train_x = np.take(x_np_scale, ids_train, axis=0)
    part_train_y = np.take(y_np, ids_train, axis=0)
    
    part_test_x = np.take(x_np_scale, ids_test, axis=0)
    part_test_y = np.take(y_np, ids_test, axis=0)
    
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    
    model_fit = model.fit(part_train_x, part_train_y)
    
    prob_scores_arr = model_fit.predict_proba(part_test_x)
    pred_class = model_fit.predict(part_test_x)
    ref_class = part_test_y
    epoch = 0
    dsettype = 'test'

    dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
                              outlog = os.path.join(exp_dir, dsettype + ".log"))
    
    predictions_df = build_predictions_df(ids_test, ref_class, pred_class, prob_scores_arr)
    predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
    
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

### Scores

In [158]:
# time_stamp = "2022-03-10_18-42-45"

import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/fold_*_"+time_stamp)
len(exp_dirs)

5

In [159]:
folds = []

for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[1]
    with open(os.path.join(edir, "test.log")) as f:
        lines = f.read().splitlines()
        folds.append([fold, float(lines[18]), float(lines[22])])

In [160]:
df_folds = pd.DataFrame(data=folds, columns=["Fold","AUPR", "AUC"]).set_index("Fold")
# df_folds["Fscore"] = df_folds.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]), axis=1) # harmonic mean of AUC, AUPR
df_folds

,AUPR,AUC
Fold,,
0,0.709244,0.809770
1,0.712646,0.811239
2,0.711489,0.812527
3,0.698664,0.807837
4,0.711675,0.810233


In [161]:
df_folds.mean(axis=0).values

array([0.70874378, 0.81032109])

In [162]:
F_score(*df_folds.mean(axis=0).values)

0.7561362836084685